In [543]:
# We import the necessary libraries
import sim           # library to connect with CoppeliaSim
import sympy as sp   # library for symbolic calculation
import numpy as np
from sympy import *
from sympy.physics.vector import init_vprinting
from sympy.physics.mechanics import dynamicsymbols
from time import sleep


In [544]:
init_vprinting(use_latex='mathjax', pretty_print=False)

In [545]:



# We define the symbolic variables
theta1, theta2, d3, lc, la, lb, theta, alpha, a, d = dynamicsymbols('theta1 theta2 d3 lc la lb theta alpha a d')
theta1, theta2, d3, lc, la, lb, theta, alpha, a, d 

def connect(port):
# returns the client number or -1 if it cannot connect
    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('127.0.0.1',port,True,True,2000,5) # Connect
    if clientID == 0: print("conectado a", port)
    else: print("no se pudo conectar")
    return clientID

def setEffector(val):
# function that triggers the end effector remotely
# val is Int with value 0 or 1 to disable or activate the final actuator.
    res,retInts,retFloats,retStrings,retBuffer=sim.simxCallScriptFunction(clientID,
        "suctionPad", sim.sim_scripttype_childscript,"setEffector",[val],[],[],"", sim.simx_opmode_blocking)
    return res

# We require the handlers for the joints, the suction cup and the suction sensor (Allows to know if the object is nearby)
clientID = connect(19999)

retCode,tip=sim.simxGetObjectHandle(clientID,'suctionPadSensor',sim.simx_opmode_blocking)
retCode,suction=sim.simxGetObjectHandle(clientID,'suctionPad',sim.simx_opmode_blocking)
retCode,joint1=sim.simxGetObjectHandle(clientID,'Joint1',sim.simx_opmode_blocking)
retCode,joint2=sim.simxGetObjectHandle(clientID,'Joint2',sim.simx_opmode_blocking)
retCode,joint3=sim.simxGetObjectHandle(clientID,'Joint3',sim.simx_opmode_blocking)
retCode, blue = sim.simxGetObjectHandle(clientID,'blue',sim.simx_opmode_blocking)
retCode, red = sim.simxGetObjectHandle(clientID,'red',sim.simx_opmode_blocking)
retCode, green = sim.simxGetObjectHandle(clientID,'green',sim.simx_opmode_blocking)
print(tip, suction, joint1, joint2, joint3, blue, red, green)

conectado a 19999
24 22 17 19 21 28 29 30


In [546]:
# We deactivate the final actuator (suction cup)
setEffector(0)

retCode,pos_red=sim.simxGetObjectPosition(clientID, red, -1, sim.simx_opmode_blocking)
retCode,pos_green=sim.simxGetObjectPosition(clientID, green, -1, sim.simx_opmode_blocking)
retCode,pos_blue=sim.simxGetObjectPosition(clientID, blue, -1, sim.simx_opmode_blocking)

q = pos_green #posición Home and Collected Prism (Suction Cup Up)


In [547]:
dest_suction_cup = pos_green
x, y, z = dest_suction_cup

# eq1 is the one that gives us the value of the position X (eq=f(x)-x=0)
eq1 = 0.2 * cos(theta1) + 0.2 * cos(theta1 + theta2) - x


# eq2 is the one that gives us the value of the position Y (eq=f(y)-y=0)
eq2 = (0.2 * sin(theta1) + 0.2 * sin(theta1 + theta2)) - y


# eq3 is the one that gives us the value of the position Z (eq=f(z)-z=0)   0.108 is the real height of manipulator
eq3 = 0.108 - d3 - z



eq1, eq2, eq3


(0.2*cos(theta1 + theta2) + 0.2*cos(theta1) - 0.299889743328094, 0.2*sin(theta1 + theta2) + 0.2*sin(theta1) - 0.0999878793954849, 0.0980000728666782 - d3)

In [548]:
q=nsolve((eq1,eq2,eq3),(theta1,theta2,d3),(q[0],q[1],q[2]))
print(q)

Matrix([[-0.337676101185551], [1.31900115157393], [0.0980000728666782]])


In [549]:
# We lower the suction cup to the object to be taken, without causing a collision (we place it aprox 1 mm above)

retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0]+0.02, sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1]+0.02, sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, -q[2]+0.01, sim.simx_opmode_oneshot)
sleep(1)
setEffector(1)
sleep(1)
retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, q[2]+0.01, sim.simx_opmode_oneshot)

In [550]:
q = pos_red #posición Home and Collected Prism (Suction Cup Up)

In [551]:
dest_suction_cup = pos_red
x, y, z = dest_suction_cup

# eq1 is the one that gives us the value of the position X (eq=f(x)-x=0)
eq1 = 0.2 * cos(theta1) + 0.2 * cos(theta1 + theta2) - x


# eq2 is the one that gives us the value of the position Y (eq=f(y)-y=0)
eq2 = (0.2 * sin(theta1) + 0.2 * sin(theta1 + theta2)) - y


# eq3 is the one that gives us the value of the position Z (eq=f(z)-z=0)   0.108 is the real height of manipulator
eq3 = 0.108 - d3 - z



eq1, eq2, eq3


(0.2*cos(theta1 + theta2) + 0.2*cos(theta1) - 0.209859549999237, 0.2*sin(theta1 + theta2) + 0.2*sin(theta1) - 0.20998315513134, 0.0980000728666782 - d3)

In [552]:
q=nsolve((eq1,eq2,eq3),(theta1,theta2,d3),(q[0],q[1],q[2]))
print(q)

Matrix([[0.0512196341539983], [1.46894587485561], [0.0980000728666782]])


In [553]:
# We lower the suction cup to the object to be taken, without causing a collision (we place it aprox 1 mm above)

retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0]-0.02, sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1]-0.02, sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, -q[2]+0.04, sim.simx_opmode_oneshot)


In [554]:
setEffector(0)


0

In [555]:

retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, q[2], sim.simx_opmode_oneshot)



In [556]:
q = pos_blue #posición Home and Collected Prism (Suction Cup Up)

In [557]:
dest_suction_cup = pos_blue
x, y, z = dest_suction_cup

# eq1 is the one that gives us the value of the position X (eq=f(x)-x=0)
eq1 = 0.2 * cos(theta1) + 0.2 * cos(theta1 + theta2) - x


# eq2 is the one that gives us the value of the position Y (eq=f(y)-y=0)
eq2 = (0.2 * sin(theta1) + 0.2 * sin(theta1 + theta2)) - y


# eq3 is the one that gives us the value of the position Z (eq=f(z)-z=0)   0.108 is the real height of manipulator
eq3 = 0.108 - d3 - z



eq1, eq2, eq3


(0.2*cos(theta1 + theta2) + 0.2*cos(theta1) - 0.219182595610619, 0.2*sin(theta1 + theta2) + 0.2*sin(theta1) - 0.0999118462204933, 0.0980000728666782 - d3)

In [558]:
q=nsolve((eq1,eq2,eq3),(theta1,theta2,d3),(q[0],q[1],q[2]))
print(q)

Matrix([[-0.496842457989044], [1.84908197856399], [0.0980000728666782]])


In [559]:
# We lower the suction cup to the object to be taken, without causing a collision (we place it aprox 1 mm above)

retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0]-0.02, sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1]-0.02, sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, -q[2]+0.01, sim.simx_opmode_oneshot)
setEffector(1)


0

In [560]:
setEffector(1)

retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, q[2], sim.simx_opmode_oneshot)



In [561]:
retCode,pos_green=sim.simxGetObjectPosition(clientID, green, -1, sim.simx_opmode_blocking)

In [562]:
q = pos_green #posición Home and Collected Prism (Suction Cup Up)


In [563]:
dest_suction_cup = pos_green
x, y, z = dest_suction_cup

# eq1 is the one that gives us the value of the position X (eq=f(x)-x=0)
eq1 = 0.2 * cos(theta1) + 0.2 * cos(theta1 + theta2) - x


# eq2 is the one that gives us the value of the position Y (eq=f(y)-y=0)
eq2 = (0.2 * sin(theta1) + 0.2 * sin(theta1 + theta2)) - y


# eq3 is the one that gives us the value of the position Z (eq=f(z)-z=0)   0.108 is the real height of manipulator
eq3 = 0.108 - d3 - z



eq1, eq2, eq3


(0.2*cos(theta1 + theta2) + 0.2*cos(theta1) - 0.197858601808548, 0.2*sin(theta1 + theta2) + 0.2*sin(theta1) - 0.214632734656334, 0.0780000733137131 - d3)

In [564]:
q=nsolve((eq1,eq2,eq3),(theta1,theta2,d3),(q[0],q[1],q[2]))
print(q)

Matrix([[0.0732613882538821], [1.50555960020247], [0.0780000733137131]])


In [565]:
# We lower the suction cup to the object to be taken, without causing a collision (we place it aprox 1 mm above)

retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, -q[2]+0.06, sim.simx_opmode_oneshot)
sleep(1)
setEffector(0)
retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0]-1, sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1]-1, sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, q[2], sim.simx_opmode_oneshot)


